### Import thư viện

In [15]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import time
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [13]:
def get_feature_importance(table, name, model, X_test, y_test, time):
    table.append([name, mean_squared_error(y_test, model.predict(X_test))/ y_test.var(), r2_score(y_test, model.predict(X_test)), time])
    return table

### Data preprocessing

In [18]:
recipes = pd.read_csv("epi_r.csv")
recipes = recipes.dropna()
recipes

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,The Best Blts,4.375,948.0,19.0,79.0,1042.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X = recipes.drop([i for i in recipes.columns[:12]], axis=1)
y = recipes['calories']
print(X.shape, y.shape)

(15864, 668) (15864,)


Dùng PCA để giảm chiều dữ liệu

In [6]:
pca = PCA(n_components=34)   # Khởi tạo PCA với 34 thành phần chính
pca.fit(X)  # Fit PCA với dữ liệu
X_pca = pca.transform(X)    # Chuyển đổi dữ liệu sang thành phần chính đầu tiên
print(X_pca.shape, y.shape)

(15864, 34) (15864,)


### Normalize

In [7]:
# Standardize the features
scaler = StandardScaler()
print("Scaling...")
X = scaler.fit_transform(X)

Scaling...


In [8]:
# Standardize the features
scaler = StandardScaler()
print("Scaling...")
X_pca_scaled = scaler.fit_transform(X_pca)

Scaling...


### Split train/test

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(12691, 668) (12691,) (3173, 668) (3173,)


In [10]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca_scaled, y, test_size=0.2, random_state=23)
print(X_pca_train.shape, y_pca_train.shape, X_pca_test.shape, y_pca_test.shape)

(12691, 34) (12691,) (3173, 34) (3173,)


### Training 

#### Trước khi dùng PCA

In [26]:
# Logistic Regression with L1 regularization
print("Training l1...")
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')
%time model_l1.fit(X_train_scaled, y_train)

Training l1...
CPU times: total: 3min 11s
Wall time: 19min 21s


LogisticRegression(penalty='l1', solver='liblinear')

In [27]:
# Logistic Regression with L2 regularization
print("Training l2...")
model_l2 = LogisticRegression(penalty='l2', solver='liblinear')
%time model_l2.fit(X_train_scaled, y_train)

Training l2...
CPU times: total: 8min 31s
Wall time: 52min 18s


LogisticRegression(solver='liblinear')

In [28]:
# Logistic Regression with Elastic Net regularization
print("Training l1 & l2...")
model_elasticnet = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
%time model_elasticnet.fit(X_train_scaled, y_train)

Training l1 & l2...
CPU times: total: 30min
Wall time: 3h 37s


c:\Users\Mink\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')

In [29]:
print("MSE...")
mse_l1 = mean_squared_error(y_test, model_l1.predict(X_test))
mse_l2 = mean_squared_error(y_test, model_l2.predict(X_test))
mse_elasticnet = mean_squared_error(y_test, model_elasticnet.predict(X_test))

print("MSE L1: %.4f" % mse_l1)
print("MSE L2: %.4f" % mse_l2)
print("MSE Elastic Net: %.4f" % mse_elasticnet)

MSE...
MSE L1: 519546.5972
MSE L2: 523598.9783
MSE Elastic Net: 510092.8393


In [31]:
print("R2 Score...")
r2_score_l1 = r2_score(y_test, model_l1.predict(X_test))
r2_score_l2 = r2_score(y_test, model_l2.predict(X_test))
r2_score_elasticnet = r2_score(y_test, model_elasticnet.predict(X_test))

print("R2 Score L1: %.4f" % r2_score_l1)
print("R2 Score L2: %.4f" % r2_score_l2)
print("R2 Score Elastic Net: %.4f" % r2_score_elasticnet)

R2 Score...
R2 Score L1: 0.1635
R2 Score L2: 0.1570
R2 Score Elastic Net: 0.1787


#### Sau khi dùng PCA

In [11]:
# Logistic Regression with L1 regularization
print("Training l1...")
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')
%time model_l1.fit(X_pca_train, y_pca_train)

Training l1...
CPU times: total: 9.72 s
Wall time: 55.2 s


LogisticRegression(penalty='l1', solver='liblinear')

In [14]:
# Logistic Regression with L2 regularization
print("Training l2...")
model_l2 = LogisticRegression(penalty='l2', solver='liblinear')
%time model_l2.fit(X_pca_train, y_pca_train)

Training l2...


In [ ]:
# Logistic Regression with Elastic Net regularization
print("Training l1 & l2...")
model_elasticnet = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
%time model_elasticnet.fit(X_pca_train, y_pca_train)

Training l1 & l2...


: 

### Đánh giá

In [10]:
table = [["Optimier", "MSE", "R2 Score", "Time"]]

In [ ]:
table = get_feature_importance(table, "Lasso_pca", model_l1, X_pca_test, y_pca_test, '54.3 s')
table = get_feature_importance(table, "Ridge_pca", model_l2, X_pca_test, y_pca_test, '1min 3s')
table = get_feature_importance(table, "ElasticNet_pca", model_elasticnet, X_pca_test, y_pca_test, '3h 37s')

NameError: name 'model_l1' is not defined

In [ ]:
pd.DataFrame(table)

,0,1,2,3
0,Optimier,MSE,R2 Score,Time
1,Lasso,0.836226,0.163511,19min 21s
2,Ridge,0.842748,0.156986,52min 18s
3,ElasticNet,0.821009,0.178732,3h 37s
